In [ ]:
# import os

# i2m = list(zip(range(1,13), ['Gener','Febrer','Marc','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))
# for year in [2023, 2022, 2021, 2020]:
#     for month, month_name in i2m:
#         os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
#         os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
#         os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

In [ ]:
!pip  install py7zr


In [ ]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import py7zr

# Create a target directory if it doesn't exist
target_directory = "bicing_data"
os.makedirs(target_directory, exist_ok=True)

i2m = list(zip(range(1, 13), ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig',
           'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']))


def download_and_extract(year, month, month_name):
    url = f"https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z"
    local_filename = f"{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z"
    try:
        # Download the file
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for request errors
        with open(local_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        # Extract the file to the target directory
        with py7zr.SevenZipFile(local_filename, mode='r') as archive:
            archive.extractall(path=target_directory)

        # Remove the downloaded .7z file
        os.remove(local_filename)
    except Exception as e:
        print(f"Failed to download or extract {local_filename}: {e}")


# Use ThreadPoolExecutor for parallel downloading
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(download_and_extract, year, month, month_name)
               for year in [2023, 2022, 2021, 2020]
               for month, month_name in i2m]

    for future in as_completed(futures):
        if future.exception() is not None:
            print(f"Generated an exception: {future.exception()}")

print("All downloads and extractions are complete.")

In [ ]:
!pip install dask


In [84]:
import dask.dataframe as dd

import pandas as pd



# Define the path to the directory containing the CSV files
csv_directory = 'bicing_data'

# Use a wildcard to read all CSV files in the directory
csv_files = f'{csv_directory}/*.csv'

# Read the CSV files into a single Dask DataFrame
# df = dd.read_csv(csv_files)

In [85]:
# Specify the dtypes for known columns to optimize reading
dtypes = {
    'station_id': 'float64',
    'num_bikes_available': 'float64',
    'num_bikes_available_types.mechanical': 'int64',
    'num_bikes_available_types.ebike': 'int64',
    'num_docks_available': 'float64',
    'is_installed': 'float64',
    'is_renting': 'object',
    'is_returning': 'bool',
    'is_charging_station': 'bool',
    'status': 'object',
    'ttl': 'float64'
}

df = dd.read_csv(csv_files, dtype=dtypes, parse_dates=[
                 'last_updated', 'last_reported'])

/Users/demian.moschin/anaconda3/lib/python3.11/site-packages/dask/dataframe/io/csv.py:629: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  head = reader(BytesIO(b_sample), nrows=sample_rows, **head_kwargs)
/Users/demian.moschin/anaconda3/lib/python3.11/site-packages/dask/dataframe/io/csv.py:629: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  head = reader(BytesIO(b_sample), nrows=sample_rows, **head_kwargs)


In [86]:
# Print the schema (dtypes) of the DataFrame
print(df.dtypes)

station_id                              float64
num_bikes_available                     float64
num_bikes_available_types.mechanical      int64
num_bikes_available_types.ebike           int64
num_docks_available                     float64
is_installed                            float64
is_renting                               object
is_returning                               bool
last_reported                            object
is_charging_station                        bool
status                                   object
last_updated                             object
ttl                                     float64
dtype: object


In [87]:
df.head(10)

/Users/demian.moschin/anaconda3/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = reader(bio, **kwargs)
/Users/demian.moschin/anaconda3/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = reader(bio, **kwargs)


,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl
0,1.0,21.0,21,0,23.0,1.0,1,True,1577833004,True,IN_SERVICE,1577833223,28.0
1,2.0,10.0,9,1,16.0,1.0,1,True,1577833078,True,IN_SERVICE,1577833223,28.0
2,3.0,15.0,15,0,9.0,1.0,1,True,1577832980,True,IN_SERVICE,1577833223,28.0
3,4.0,8.0,8,0,13.0,1.0,1,True,1577833130,True,IN_SERVICE,1577833223,28.0
4,5.0,14.0,11,3,24.0,1.0,1,True,1577833130,True,IN_SERVICE,1577833223,28.0
5,6.0,6.0,6,0,33.0,1.0,1,True,1577832965,True,IN_SERVICE,1577833223,28.0
6,7.0,8.0,8,0,19.0,1.0,1,True,1577833085,True,IN_SERVICE,1577833223,28.0
7,8.0,18.0,18,0,8.0,1.0,1,True,1577832998,True,IN_SERVICE,1577833223,28.0
8,9.0,22.0,20,2,2.0,1.0,1,True,1577832972,True,IN_SERVICE,1577833223,28.0
9,11.0,0.0,0,0,21.0,1.0,1,True,1577833083,True,IN_SERVICE,1577833223,28.0


In [89]:

df.tail(n=5, compute=True)

/Users/demian.moschin/anaconda3/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = reader(bio, **kwargs)
/Users/demian.moschin/anaconda3/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = reader(bio, **kwargs)


,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl
515,5.0,5.0,0,19,1.704064e+09,1.0,IN_SERVICE,True,1,True,NaN,1704064502,1.0
516,10.0,2.0,8,10,1.704064e+09,1.0,IN_SERVICE,True,1,True,NaN,1704064502,1.0
517,8.0,1.0,7,11,1.704064e+09,1.0,IN_SERVICE,True,1,True,NaN,1704064502,1.0
518,3.0,1.0,2,24,1.704064e+09,1.0,IN_SERVICE,True,1,True,NaN,1704064502,1.0
519,6.0,0.0,6,17,1.704064e+09,1.0,IN_SERVICE,True,1,True,NaN,1704064502,1.0
